In [ ]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math

from DataCollapse import DataCollapse

In [ ]:
num_unit_cell_vals = [8, 16, 32, 64]
N_val = 4
num_init_conds = 1000
a_vals = [0.763]

In [ ]:
def load_decay_data(path):
    
    data_dict = {}

    for L_val in [num_unit_cell_val * N_val for num_unit_cell_val in num_unit_cell_vals]:
        fname = f"lambda_var_per_t_N4_ar{str(min(a_vals)).replace(".", "p")}_{str(max(a_vals)).replace(".", "p")}_IC1000_L{int(L_val)}_AW{avraging_window_name}.csv"
        df = pd.read_csv(os.path.join(path, fname))

        for _, row in df.iterrows():
            a = row['a']
            observations = row['observations']
            data_dict[(a, L_val)] = np.array([observations for _ in range(0, 10)])

    # build MultiIndex DataFrame
    index = pd.MultiIndex.from_tuples(data_dict.keys(), names=['a','L'])
    df = pd.DataFrame({'observations': list(data_dict.values())}, index=index)
    return df

decay_df = load_decay_data(f"../../data_to_collapse/lambda_var_per_a/N{N_val}/SeveralAs/IC{num_init_conds}/SeveralLs/").sort_index(level='a').rename_axis(index={'a': 'p'})